<a href="https://colab.research.google.com/github/0ginny/TIL/blob/main/240605_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%88%98%EC%9A%94%EC%98%88%EC%B8%A1_%EC%98%88%EC%B8%A1%EC%84%9C%EB%B2%84%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# pickle 형태 model 불러오기
import pickle
# 행과 열을 다루기
import pandas as pd
import numpy as np

In [32]:
# 서버 관리용 fastapi 의존 라이브러리
# !pip install uvicorn
import uvicorn

In [33]:
# !pip install fastapi
from fastapi import FastAPI

In [34]:
# 인터페이스 데이터 관리
from pydantic import BaseModel

In [35]:
# crossover(보안문제) 해결 라이브러리
from fastapi.middleware.cors import CORSMiddleware

In [52]:
origins = ["*"]
app = FastAPI(title="GINNY ML API")
# CORS 미들웨어 추가
app.add_middleware(
CORSMiddleware,
allow_origins=["*"], # 모든 origin 허용
allow_credentials=True,
allow_methods=["GET", "POST", "PUT", "DELETE"], # 모든 방식 다 사용
allow_headers=["*"],
)

In [61]:
# !pip install nest-asyncio pyngrok
import nest_asyncio
from pyngrok import ngrok
import uvicorn

## 0. 입력 클래스 선언

In [54]:
# 변경포인트
class InDataset(BaseModel):
    inHclus : int
    inPropercent : float
    inPromotion : str
    inHoliday : str

## 1. 학습모델 불러오기

In [55]:
with open("240605_first_QTY_regretion.dump",'rb') as fr:
    loadedDump = pickle.load(fr)

In [56]:
features = loadedDump["features"]
label = loadedDump["label"]
preprocessing = loadedDump["preprocessing"]
loadedModel = loadedDump["model"]

## 2. 예측 시뮬레이션

In [63]:
@app.post("/predict",status_code=200) # 신호가 정상일때만 작동. ip:port/predict 될때 작동.
async def predictSellout(x:InDataset) -> dict: # inDataset 형식인지 적합성 평가
    inHclus = x.inHclus
    inPropercent = x.inPropercent
    inHoliday = x.inHoliday
    inPromotion = x.inPromotion

    # 전처리
    inHolidayConv = preprocessing[2].transform([inHoliday])[0]
    inPromotionConv = preprocessing[3].transform([inPromotion])[0]
    inDf = pd.DataFrame([[inHclus,inPropercent,inHolidayConv,inPromotionConv]])
    predictValue = loadedModel.predict(inDf)[0]
    result = {'prediction' : predictValue}
    return result

In [64]:
# 작동 확인용 코드
@app.get("/")
async def root():
    return {"message":"server is running"}

In [ ]:
# 서버 작동
auth_token = "MY_TOKEN"
ngrok.set_auth_token(auth_token)
ngrokTunnel = ngrok.connect(8000)
print("공용 URL", ngrokTunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)